In [617]:
import pandas as pd
from tqdm import tqdm
import warnings
import numpy as np
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor
import sklearn as sk

warnings.filterwarnings('ignore')

In [618]:
df = pd.read_csv("files\\pred_final.csv")
df

,obj_prg,obj_subprg,obj_key,Кодзадачи,ПроцентЗавершенияЗадачи,ДатаНачалаЗадачи,ДатаОкончанияЗадачи,Статуспоэкспертизе,Экспертиза,date_report,season,Скорость,Кол-во рабочих,Генподрядчик,Генпроектировщик,Время на выполнение,Площадь
0,0,0,022-0527,1,0,2022-01-14,2023-03-30,0,0,2023-01-17,1,0.257576,0.0,0.0,0.0,440,9850.0
1,0,0,022-0354,1,0,2022-12-01,2023-07-27,0,0,2023-01-17,1,0.462121,0.0,0.0,0.0,238,20271.0
2,0,1,022-0513,1,0,2023-01-23,2023-09-29,0,0,2023-01-17,1,0.416667,0.0,0.0,0.0,249,6892.0
3,0,1,020-0684,1,0,2020-11-03,2022-02-01,0,0,2023-01-17,4,0.000000,0.0,2.0,1.0,455,2488.6
4,0,1,019-0589,1,0,2020-11-03,2022-05-16,0,0,2023-01-17,4,0.000000,0.0,2.0,1.0,559,3751.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35196,1,2,019-0477,7.4,93,2020-04-28,2023-05-31,0,0,2023-05-29,2,0.704545,5.0,12.0,16.0,1128,0.0
35197,0,0,020-0710,8,0,2023-08-31,2024-04-10,0,0,2023-05-29,3,0.000000,156.0,5.0,8.0,223,0.0
35198,0,0,019-0675,8,0,2023-09-01,2024-04-11,0,0,2023-05-29,4,0.000000,139.0,22.0,33.0,223,0.0
35199,0,1,020-0712,8,0,2023-06-19,2024-01-26,0,0,2023-05-29,2,0.000000,53.0,5.0,8.0,221,0.0


In [619]:
date_reports = df['date_report'].unique()
columns = ['obj_key', 'obj_subprg', 'Кодзадачи', 'ДатаНачалаЗадачи', 'Статуспоэкспертизе', 'Экспертиза', 'season',
           'Скорость', 'Кол-во рабочих', 'Генподрядчик', 'Генпроектировщик', 'Площадь', *date_reports, 'y']
columns

['obj_key',
 'obj_subprg',
 'Кодзадачи',
 'ДатаНачалаЗадачи',
 'Статуспоэкспертизе',
 'Экспертиза',
 'season',
 'Скорость',
 'Кол-во рабочих',
 'Генподрядчик',
 'Генпроектировщик',
 'Площадь',
 '2023-01-17',
 '2023-01-23',
 '2023-02-06',
 '2023-02-13',
 '2023-02-20',
 '2023-02-27',
 '2023-03-06',
 '2023-03-13',
 '2023-03-20',
 '2023-03-27',
 '2023-04-03',
 '2023-04-10',
 '2023-04-17',
 '2023-04-24',
 '2023-05-02',
 '2023-05-10',
 '2023-05-15',
 '2023-05-22',
 '2023-05-29',
 'y']

In [620]:
df = df.sort_values(['obj_key', 'Кодзадачи', 'ПроцентЗавершенияЗадачи', 'date_report'])

In [621]:
len(date_reports)

19

In [622]:
d_dates = {key: np.nan for key in date_reports}
list(d_dates.values())

[nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan]

In [623]:
new_df = pd.DataFrame(columns=columns)
keys = df['obj_key'].unique()
keys.sort()
# pd.Series(row_data, index=columns
for key in tqdm(keys):
    codes = df[df['obj_key'] == key]['Кодзадачи'].unique()
    for code in codes:
        objects = df[(df['obj_key'] == key) & (df['Кодзадачи'] == code)]
        start = objects.head(1)
        last = objects.tail(1)
        obj_key = key
        obj_subprg = last['obj_subprg'].values[0]
        obj_code = code
        start_date = start['ДатаНачалаЗадачи'].values[0]
        status_exp = last['Статуспоэкспертизе'].values[0]
        exp = last['Экспертиза'].values[0]
        season = last['season'].values[0]
        speed = last['Скорость'].values[0]
        rab_count = start['Кол-во рабочих'].values[0]
        gen_pod = last['Генподрядчик'].values[0]
        gen_proc = last['Генпроектировщик'].values[0]
        square = last['Площадь'].values[0]
        y_value = last['Время на выполнение'].values[0]
        dates = objects['date_report'].unique()
        d_dates = {key: np.nan for key in date_reports}
        row_data = [obj_key, obj_subprg, obj_code, start_date, status_exp, exp, season, speed, rab_count, gen_pod,
                    gen_proc, square]
        for date in dates:
            prc = objects[objects['date_report'] == date].head(1)['ПроцентЗавершенияЗадачи'].values[0]
            d_dates[date] = prc
        row_data.extend(list(d_dates.values()))
        row_data.append(y_value)
        new_df = new_df.append(pd.Series(row_data, index=columns), ignore_index=True)
new_df

100%|██████████| 72/72 [00:19<00:00,  3.69it/s]


,obj_key,obj_subprg,Кодзадачи,ДатаНачалаЗадачи,Статуспоэкспертизе,Экспертиза,season,Скорость,Кол-во рабочих,Генподрядчик,...,2023-04-03,2023-04-10,2023-04-17,2023-04-24,2023-05-02,2023-05-10,2023-05-15,2023-05-22,2023-05-29,y
0,017-0520,1,1,2018-11-01,0,0,4,0.757576,0.0,5.0,...,100,100,100,100,100,100,100,100,100,1202
1,017-0520,1,1.4.2,2021-12-15,0,0,1,0.757576,0.0,5.0,...,100,100,100,100,100,100,100,100,100,62
2,017-0520,1,3,2021-02-20,0,0,2,0.083333,0.0,5.0,...,11,11,11,11,11,11,11,11,11,373
3,017-0520,1,3.1,2021-02-20,0,0,2,0.681818,0.0,5.0,...,38,38,38,38,38,38,38,38,38,220
4,017-0520,1,3.11,2021-04-20,0,0,2,0.000000,0.0,5.0,...,0,0,0,0,0,0,0,0,0,314
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1840,022-0631,3,5.11,2024-10-01,0,0,4,0.000000,0.0,2.0,...,0,0,0,0,0,0,0,0,0,4
1841,022-0631,3,7.1,2024-10-01,0,0,3,0.000000,0.0,2.0,...,0,0,0,0,0,0,0,0,0,44
1842,022-0631,3,7.1.8,2024-11-11,0,0,4,0.000000,0.0,2.0,...,0,0,0,0,0,0,0,0,0,3
1843,022-0631,3,7.4,2024-11-12,0,0,4,0.000000,0.0,2.0,...,0,0,0,0,0,0,0,0,0,28


In [624]:
def find_good_idx(vals: np.ndarray, start_index: int = 12) -> tuple[int, bool]:
    good_idx = 0
    for i in range(start_index, len(vals) - 2):
        if not np.isnan(vals[i]):
            good_idx = i
    if good_idx == 0:
        return 0, False
    else:
        return good_idx, True

In [625]:
for i, row in new_df.iterrows():
    has_nan = row.isna().any()
    if has_nan:
        vals = row.values
        for j in range(12, len(vals) - 1):
            if np.isnan(vals[j]):
                good_idx, ans = find_good_idx(vals, j)
                if ans:
                    vals[j] = vals[good_idx]
                else:
                    good_idx, ans = find_good_idx(vals, 12)
                    vals[j] = vals[good_idx]
        new_df.loc[i] = vals

In [626]:
new_df['Генпроектировщик'] = new_df['Генпроектировщик'].astype(int)
new_df['Генподрядчик'] = new_df['Генподрядчик'].astype(int)
new_df['Кол-во рабочих'] = new_df['Кол-во рабочих'].astype(int)
new_df['ДатаНачалаЗадачи'] = pd.to_datetime(new_df['ДатаНачалаЗадачи']).apply(lambda x: x.toordinal())
new_df['obj_subprg'] = new_df['obj_subprg'].astype(int)
new_df['Статуспоэкспертизе'] = new_df['Статуспоэкспертизе'].astype(int)
new_df['Экспертиза'] = new_df['Экспертиза'].astype(int)
new_df['season'] = new_df['season'].astype(int)
new_df['y'] = new_df['y'].astype(int)
for column in date_reports:
    new_df[column] = new_df[column].astype(int)
new_df.dtypes

obj_key                object
obj_subprg              int32
Кодзадачи              object
ДатаНачалаЗадачи        int64
Статуспоэкспертизе      int32
Экспертиза              int32
season                  int32
Скорость              float64
Кол-во рабочих          int32
Генподрядчик            int32
Генпроектировщик        int32
Площадь               float64
2023-01-17              int32
2023-01-23              int32
2023-02-06              int32
2023-02-13              int32
2023-02-20              int32
2023-02-27              int32
2023-03-06              int32
2023-03-13              int32
2023-03-20              int32
2023-03-27              int32
2023-04-03              int32
2023-04-10              int32
2023-04-17              int32
2023-04-24              int32
2023-05-02              int32
2023-05-10              int32
2023-05-15              int32
2023-05-22              int32
2023-05-29              int32
y                       int32
dtype: object

In [627]:
cat_features = ["Кодзадачи", 'obj_subprg', 'Статуспоэкспертизе', 'Экспертиза', 'season', 'Генподрядчик',
                'Генпроектировщик']

In [628]:
new_df.to_csv("files\\times.csv", index=False)

In [629]:
import pickle

encoder = sk.preprocessing.OneHotEncoder()
encoded_features = encoder.fit_transform(new_df[cat_features])
with open("encoder.pkl", "wb") as f:
    pickle.dump(encoder, f)
encoded_df = pd.DataFrame(encoded_features.toarray()).reset_index(drop=True)
df_no_cat = new_df.drop(cat_features, axis=1).reset_index(drop=True)
new_data = pd.concat([encoded_df, df_no_cat], axis=1)

new_data

,0,1,2,3,4,5,6,7,8,9,...,2023-04-03,2023-04-10,2023-04-17,2023-04-24,2023-05-02,2023-05-10,2023-05-15,2023-05-22,2023-05-29,y
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,100,100,100,100,100,100,100,100,100,1202
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,100,100,100,100,100,100,100,100,100,62
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,11,11,11,11,11,11,11,11,11,373
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,38,38,38,38,38,38,38,38,38,220
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,314
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1840,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,4
1841,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,44
1842,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,3
1843,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,28


In [630]:
new_df3 = new_df.copy()
new_df.drop(cat_features, inplace=True, axis=1)
new_data.columns = [str(col) for col in
                    new_data.columns]  # добавляем префикс к имени каждой колонки, чтобы избежать дубликатов
new_df2 = new_data.copy()
new_df2

,0,1,2,3,4,5,6,7,8,9,...,2023-04-03,2023-04-10,2023-04-17,2023-04-24,2023-05-02,2023-05-10,2023-05-15,2023-05-22,2023-05-29,y
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,100,100,100,100,100,100,100,100,100,1202
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,100,100,100,100,100,100,100,100,100,62
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,11,11,11,11,11,11,11,11,11,373
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,38,38,38,38,38,38,38,38,38,220
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,314
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1840,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,4
1841,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,44
1842,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,3
1843,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,28


In [631]:
len(new_df)

1845

In [632]:
x_train, x_test, y_train, y_test = train_test_split(new_df2.drop(['obj_key', 'y'], axis=1), new_df2['y'],
                                                    test_size=0.28,
                                                    random_state=228)

In [633]:
# leaf_estimation_backtracking="AnyImprovement"
model = CatBoostRegressor(iterations=15000, loss_function='MAE', learning_rate=0.05, max_depth=5, eval_metric="MAE",
                          leaf_estimation_backtracking="AnyImprovement", subsample=0.7, bagging_temperature=0.3,
                          langevin=True)

In [634]:
model.fit(x_train, y_train, use_best_model=True, eval_set=(x_test, y_test))

0:	learn: 178.5753755	test: 176.1021266	best: 176.1021266 (0)	total: 1.64ms	remaining: 24.6s
1:	learn: 174.8159175	test: 172.3443401	best: 172.3443401 (1)	total: 3.01ms	remaining: 22.6s
2:	learn: 171.3302864	test: 168.7118823	best: 168.7118823 (2)	total: 4.34ms	remaining: 21.7s
3:	learn: 168.5900589	test: 165.8852795	best: 165.8852795 (3)	total: 5.64ms	remaining: 21.2s
4:	learn: 165.6855414	test: 163.1131498	best: 163.1131498 (4)	total: 7.03ms	remaining: 21.1s
5:	learn: 163.2149003	test: 160.5165013	best: 160.5165013 (5)	total: 8.54ms	remaining: 21.3s
6:	learn: 161.3451175	test: 158.5927928	best: 158.5927928 (6)	total: 10.3ms	remaining: 22.1s
7:	learn: 159.4227211	test: 156.4159785	best: 156.4159785 (7)	total: 11.8ms	remaining: 22.1s
8:	learn: 157.3281353	test: 154.2119055	best: 154.2119055 (8)	total: 13.3ms	remaining: 22.2s
9:	learn: 155.8852314	test: 152.7156698	best: 152.7156698 (9)	total: 15ms	remaining: 22.4s
10:	learn: 153.8924702	test: 150.4005388	best: 150.4005388 (10)	total: 1

In [635]:
model.get_feature_importance(prettified=True)

,Feature Id,Importances
0,ДатаНачалаЗадачи,25.753658
1,2,8.834230
2,9,6.007446
3,0,3.522529
4,Кол-во рабочих,3.303699
...,...,...
119,87,0.002737
120,100,0.001028
121,35,0.000000
122,58,0.000000


In [636]:
from sklearn.metrics import mean_absolute_error

mean_absolute_error(y_test, model.predict(x_test))

63.54760507255228

In [637]:
vals = new_df['y'].unique()
vals.sort()
print(min(vals), max(vals))


0 1547


In [638]:
len(new_df)
model.save_model('cat_model')

In [639]:
len(model.predict(x_test))

517

In [640]:
new_df3 = new_df3[:517]
new_df3['yy'] = model.predict(x_test)


In [641]:
new_df3['yy'] = new_df3['yy'].apply(lambda x: 0 if x < 0 else x)
new_df3['yy'] = new_df3['yy'].round().astype(int)
new_df3

,obj_key,obj_subprg,Кодзадачи,ДатаНачалаЗадачи,Статуспоэкспертизе,Экспертиза,season,Скорость,Кол-во рабочих,Генподрядчик,...,2023-04-10,2023-04-17,2023-04-24,2023-05-02,2023-05-10,2023-05-15,2023-05-22,2023-05-29,y,yy
0,017-0520,1,1,736999,0,0,4,0.757576,0,5,...,100,100,100,100,100,100,100,100,1202,0
1,017-0520,1,1.4.2,738139,0,0,1,0.757576,0,5,...,100,100,100,100,100,100,100,100,62,130
2,017-0520,1,3,737841,0,0,2,0.083333,0,5,...,11,11,11,11,11,11,11,11,373,190
3,017-0520,1,3.1,737841,0,0,2,0.681818,0,5,...,38,38,38,38,38,38,38,38,220,247
4,017-0520,1,3.11,737900,0,0,2,0.000000,0,5,...,0,0,0,0,0,0,0,0,314,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
512,019-0477,2,4,737091,0,0,2,0.757576,95,12,...,99,99,99,100,100,100,100,100,1547,0
513,019-0477,2,4.10,738410,0,0,4,0.757576,95,12,...,100,100,100,100,100,100,100,100,184,529
514,019-0477,2,4.11,737091,0,0,2,0.757576,95,12,...,100,100,100,100,100,100,100,100,1419,133
515,019-0477,2,4.12,738361,0,0,3,0.757576,95,12,...,97,97,98,100,100,100,100,100,277,246


In [642]:
import datetime


new_df3['ДатаНачалаЗадачи'] = new_df3['ДатаНачалаЗадачи'].apply(lambda x: datetime.date.fromordinal(x))
new_df3

,obj_key,obj_subprg,Кодзадачи,ДатаНачалаЗадачи,Статуспоэкспертизе,Экспертиза,season,Скорость,Кол-во рабочих,Генподрядчик,...,2023-04-10,2023-04-17,2023-04-24,2023-05-02,2023-05-10,2023-05-15,2023-05-22,2023-05-29,y,yy
0,017-0520,1,1,2018-11-01,0,0,4,0.757576,0,5,...,100,100,100,100,100,100,100,100,1202,0
1,017-0520,1,1.4.2,2021-12-15,0,0,1,0.757576,0,5,...,100,100,100,100,100,100,100,100,62,130
2,017-0520,1,3,2021-02-20,0,0,2,0.083333,0,5,...,11,11,11,11,11,11,11,11,373,190
3,017-0520,1,3.1,2021-02-20,0,0,2,0.681818,0,5,...,38,38,38,38,38,38,38,38,220,247
4,017-0520,1,3.11,2021-04-20,0,0,2,0.000000,0,5,...,0,0,0,0,0,0,0,0,314,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
512,019-0477,2,4,2019-02-01,0,0,2,0.757576,95,12,...,99,99,99,100,100,100,100,100,1547,0
513,019-0477,2,4.10,2022-09-12,0,0,4,0.757576,95,12,...,100,100,100,100,100,100,100,100,184,529
514,019-0477,2,4.11,2019-02-01,0,0,2,0.757576,95,12,...,100,100,100,100,100,100,100,100,1419,133
515,019-0477,2,4.12,2022-07-25,0,0,3,0.757576,95,12,...,97,97,98,100,100,100,100,100,277,246


In [643]:
new_df3['kkk'] = new_df3['ДатаНачалаЗадачи'] + pd.to_timedelta(new_df3['yy'], unit='d')
new_df3

,obj_key,obj_subprg,Кодзадачи,ДатаНачалаЗадачи,Статуспоэкспертизе,Экспертиза,season,Скорость,Кол-во рабочих,Генподрядчик,...,2023-04-17,2023-04-24,2023-05-02,2023-05-10,2023-05-15,2023-05-22,2023-05-29,y,yy,kkk
0,017-0520,1,1,2018-11-01,0,0,4,0.757576,0,5,...,100,100,100,100,100,100,100,1202,0,2018-11-01
1,017-0520,1,1.4.2,2021-12-15,0,0,1,0.757576,0,5,...,100,100,100,100,100,100,100,62,130,2022-04-24
2,017-0520,1,3,2021-02-20,0,0,2,0.083333,0,5,...,11,11,11,11,11,11,11,373,190,2021-08-29
3,017-0520,1,3.1,2021-02-20,0,0,2,0.681818,0,5,...,38,38,38,38,38,38,38,220,247,2021-10-25
4,017-0520,1,3.11,2021-04-20,0,0,2,0.000000,0,5,...,0,0,0,0,0,0,0,314,0,2021-04-20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
512,019-0477,2,4,2019-02-01,0,0,2,0.757576,95,12,...,99,99,100,100,100,100,100,1547,0,2019-02-01
513,019-0477,2,4.10,2022-09-12,0,0,4,0.757576,95,12,...,100,100,100,100,100,100,100,184,529,2024-02-23
514,019-0477,2,4.11,2019-02-01,0,0,2,0.757576,95,12,...,100,100,100,100,100,100,100,1419,133,2019-06-14
515,019-0477,2,4.12,2022-07-25,0,0,3,0.757576,95,12,...,97,98,100,100,100,100,100,277,246,2023-03-28


In [644]:
df

,obj_prg,obj_subprg,obj_key,Кодзадачи,ПроцентЗавершенияЗадачи,ДатаНачалаЗадачи,ДатаОкончанияЗадачи,Статуспоэкспертизе,Экспертиза,date_report,season,Скорость,Кол-во рабочих,Генподрядчик,Генпроектировщик,Время на выполнение,Площадь
8,0,1,017-0520,1,100,2018-11-01,2022-02-15,0,0,2023-01-17,4,0.757576,0.0,5.0,8.0,1202,6367.0
1699,0,1,017-0520,1,100,2018-11-01,2022-02-15,0,0,2023-01-23,4,0.757576,0.0,5.0,8.0,1202,6367.0
3432,0,1,017-0520,1,100,2018-11-01,2022-02-15,0,0,2023-02-06,4,0.757576,0.0,5.0,8.0,1202,6367.0
5219,0,1,017-0520,1,100,2018-11-01,2022-02-15,0,0,2023-02-13,4,0.757576,0.0,5.0,8.0,1202,0.0
7030,0,1,017-0520,1,100,2018-11-01,2022-02-15,0,0,2023-02-20,4,0.757576,0.0,5.0,8.0,1202,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33242,1,3,022-0631,7.4,0,2024-10-14,2024-11-11,0,0,2023-05-22,4,0.000000,0.0,2.0,1.0,28,0.0
35153,1,3,022-0631,7.4,0,2024-10-14,2024-11-11,0,0,2023-05-29,4,0.000000,0.0,2.0,1.0,28,0.0
1665,1,3,022-0631,8,0,2024-12-02,2025-07-11,0,0,2023-01-17,1,0.000000,0.0,2.0,0.0,221,0.0
3393,1,3,022-0631,8,0,2024-12-02,2025-07-11,0,0,2023-01-23,1,0.000000,0.0,2.0,0.0,221,0.0
